In [1]:
import pandas as pd
import uuid
#from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

# Used in FL
import nest_asyncio
nest_asyncio.apply()
import collections
import tensorflow_federated as tff
import tensorflow as tf

np.random.seed(0)
import random

2023-04-19 16:07:08.635440: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 16:07:08.835666: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 16:07:08.836984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 16:07:14.285516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Testing TFF
# tff.backends.native.set_local_python_execution_context()
print(tff.federated_computation(lambda: 'Hello, World!')())

2023-04-19 16:07:29.742733: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-19 16:07:29.744123: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
E0419 16:07:29.990032300     121 socket_utils_common_posix.cc:221] check for SO_REUSEPORT: {"created":"@1681934849.990002200","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":199,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
E0419 16:07:29.990884100     121 socket_utils_common_posix.cc:327] setsockopt(TCP_USER_TIMEOUT) Protocol not available


b'Hello, World!'


In [3]:
# Read the CSV files into separate dataframes
df_ey = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_younger.csv')
df_eo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_older.csv')
df_ly = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_younger.csv')
df_lo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_older.csv')

In [4]:
# Concatenate the dataframes into one dataframe with matching headers
df = pd.concat([df_ey, df_eo, df_ly, df_lo], ignore_index=True, sort=False)

In [5]:
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
0,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
1,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
2,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
3,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
4,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
870,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
871,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0
872,1,1,3,2,1,1,48.0,0.565451,13.000000,0.310525,191.393162,3.060358,1.0,NoDx,55.0


In [6]:
# Generate a unique client ID for each row
df['client_id'] = [str(uuid.uuid4()) for _ in range(len(df))]

# Set the index of the dataframe to the client_id column
df.set_index('client_id', inplace=True)

In [7]:
# Print the dataframe
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
client_id,,,,,,,,,,,,,,,
d2bf6732-3887-4152-bf8e-156e9822b6a7,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
ebb3cee8-f484-4ec1-9771-64e146049dcc,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
4365841a-c38d-4c79-ae7a-fc258e8af2d6,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
84bd7065-5b38-41a9-a423-5151c092eb45,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
e3160f4a-7333-45f5-b761-03551eafd212,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b8089baa-8ed5-4af6-b336-b333543ffd03,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
838925dd-93a6-4217-b0d0-c4c5ece8563d,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
e0925831-0426-4192-9cd5-be499c5d5757,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0


In [8]:
# Get the columns with NaN values
nan_cols = df.columns[df.isnull().any()]

# Print the columns with NaN values as a list
print(nan_cols.tolist())

['GramComplex']


In [9]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# Fill missing values in the numeric columns with the mean value
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [10]:
# Replace the 'NoDx' value with 0 and the 'Dx' value with 1
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].replace("NoDx", 0).replace("Dx", 1)
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].astype(int)

In [11]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
print(len(train_data), len(test_data), type(train_data))

699 175 <class 'pandas.core.frame.DataFrame'>


In [13]:
print(test_data.dtypes)

Level.gender             int64
Level.ethnic             int64
Level.med                int64
Level.fed                int64
Level.income             int64
FamHist                  int64
Percentile             float64
GCC                    float64
Degree                 float64
HC                     float64
Between                float64
PathLength             float64
CombWords              float64
AnyLangorReadDxOnly      int64
GramComplex            float64
dtype: object


In [14]:
# Convert the training data to a TensorFlow Dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="AnyLangorReadDxOnly")

print(train_ds)

<_PrefetchDataset element_spec=({'Level.gender': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.ethnic': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.med': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.fed': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.income': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'FamHist': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Percentile': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'GCC': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Degree': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'HC': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Between': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'PathLength': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'CombWords': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'GramComplex': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}, TensorSpec(shape=(N

In [15]:
# ----------------------------------------------------Federated Data-----------------------------------------------------------

In [158]:
#create a list of client_names
client_names = df.index.values

# Get a numpy array of each client's values for all collumns, except the label column
client_data = df[["Level.gender", "Level.ethnic", "Level.med", "Level.fed", "Level.income", "FamHist",\
                 "Percentile", "GCC", "Degree", "HC", "Between", "PathLength", "CombWords", "GramComplex"]].values

# Get a numpy array of each client's value for only the label column
client_labels = df["AnyLangorReadDxOnly"].values

# Get an array of a tuple(client_data element, respective client_label)
data = list(zip(client_data, client_labels))

#shard data and place at each client
shards = [data[i:i + 1] for i in range(0, len(data), 1)]

# Assert number of clients must equal number of shards
assert(len(shards) == len(client_names))



In [170]:
# Create a dictionary with the client names as keys and the data columns as the values
clients = {client_names[i] : shards[i] for i in range(len(client_names))}
clients = {k: v[0] for k, v in clients.items()}
print(clients)

{'d2bf6732-3887-4152-bf8e-156e9822b6a7': (array([ 0.        ,  4.        ,  4.        ,  4.        ,  3.        ,
        0.        , 67.        ,  0.43181818,  3.63265306,  0.25874433,
       27.06122449,  2.87023977,  2.        , 58.05600077]), 0), 'ebb3cee8-f484-4ec1-9771-64e146049dcc': (array([  1.        ,   4.        ,   6.        ,   6.        ,
         3.        ,   0.        ,  95.        ,   0.59795018,
        12.48      ,   0.33605145, 144.48      ,   2.98757959,
         1.        ,  58.05600077]), 0), '4365841a-c38d-4c79-ae7a-fc258e8af2d6': (array([ 1.        ,  4.        ,  5.        ,  4.        ,  2.        ,
        1.        , 10.        ,  0.        ,  1.        ,  0.41666667,
        0.25      ,  1.33333333,  1.        , 58.05600077]), 0), '84bd7065-5b38-41a9-a423-5151c092eb45': (array([ 0.        ,  4.        ,  3.        ,  3.        ,  2.        ,
        0.        , 37.        ,  0.47368421,  2.        ,  0.21245421,
        1.14285714,  1.61538461,  1.       

In [320]:
# Batch the data_shard into a an array of tensorflow dataset
def batch_data(data_shard, bs=32, pref=25):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    def batch_format_fn(element):
        print(element['AnyLangorReadDxOnly'])
        return collections.OrderedDict(
            x=tf.reshape(element[["Level.gender", "Level.ethnic", "Level.med", "Level.fed", "Level.income", "FamHist",\
                     "Percentile", "GCC", "Degree", "HC", "Between", "PathLength", "CombWords", "GramComplex"]], [-1, 14]),
            y=tf.reshape(element['AnyLangorReadDxOnly'], (-1, 1)))#element['label'])
    #seperate shard into data and labels lists
#     data, label = zip(*data_shard)
#     dataset = tf.data.Dataset.from_tensor_slices((list(data_shard[0]), list(data_shard[1])))
    dataset = tf.data.Dataset.from_tensor_slices(collections.OrderedDict(
        x=tf.constant([data_shard[0]], dtype=tf.float64),
        y=tf.constant([[data_shard[1]]], dtype=tf.int64)
    ))
    # Shuffle the dataset, batch the 
#     dataset = dataset.map(batch_format_fn)#.repeat(5).shuffle(len(label)).batch(bs).prefetch(pref)
    return dataset#.shuffle(20).repeat(5).batch(bs).prefetch(pref)

# Run the function for each row in the clients dictionary, and add it back to a dictionary clients_batched
federated_data = []
for (client_name, data) in clients.items():
    federated_data.append(batch_data(data))
    
print(federated_data[0].element_spec)

OrderedDict([('x', TensorSpec(shape=(14,), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(1,), dtype=tf.int64, name=None))])


In [321]:
# -----------------------------------------------Create a Federated Model-------------------------------------------------------

In [322]:
# iterate over the dataset elements and print them
for x,y in federated_data[0]:
    print(x)

x


2023-04-19 18:21:00.528272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1,1]
	 [[{{node Placeholder/_1}}]]


In [323]:
def create_keras_model():
  return tfdf.keras.RandomForestModel(num_trees=50, max_depth=10)


In [324]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_data[0].element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

In [331]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

TypeError: build_weighted_fed_avg() missing 1 required positional argument: 'client_optimizer_fn'

In [326]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<>,
    non_trainable=<
      bool
    >
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [327]:
train_state = training_process.initialize()
print(train_state)

LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[], non_trainable=[False]), distributor=(), client_work=(), aggregator=OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())]), finalizer=[0])


In [328]:

result = training_process.next(train_state, federated_data)
train_state = result.state
train_metrics = result.metrics

2023-04-19 18:21:03.524650: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-19 18:21:03.525364: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-19 18:21:03.567597: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-19 18:21:03.568356: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-19 18:21:03.577248: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-19 18:21:03.577851: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-19 18:21:03.588801: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-19 18:21:03.589269: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


In [329]:
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.9416476), ('loss', 0.90008265), ('num_examples', 874), ('num_batches', 874)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [333]:
# For our implementation, it would be a challenge to pick randomly selected datasets, to simulate real world interactions.

NUM_ROUNDS = 11
# Just running the above Federated Computation 11 times, using .next()
# As you can see the system converges to a higher level of accuracy with each round.
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_data[:100])
  train_state = result.state
  train_metrics = result.metrics
  print(train_state)
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[], non_trainable=[False]), distributor=(), client_work=(), aggregator=OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())]), finalizer=[0])
round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.96), ('loss', 0.61699796), ('num_examples', 100), ('num_batches', 100)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[], non_trainable=[False]), distributor=(), client_work=(), aggregator=OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())]), finalizer=[0])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.96), ('loss', 0.61699796), ('num_examples', 100), ('num_batches', 100)]))])), ('aggregator', Or

In [ ]:
#--------------------------------------------Rest of Centralized Model----------------------------------------------------------

In [47]:
model = tfdf.keras.RandomForestModel(num_trees=50, max_depth=10)

Use /tmp/tmpejj1q6i0 as temporary training directory


In [48]:
# Fit the model to the training data
model.fit(train_ds)

Reading training dataset...


2023-04-19 16:16:10.290552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype double and shape [699]
	 [[{{node Placeholder/_13}}]]


Training dataset read in 0:00:02.929152. Found 699 examples.
Training model...
Model trained in 0:00:00.096414
Compiling model...


[INFO 23-04-19 16:16:13.2702 EDT kernel.cc:1242] Loading model from path /tmp/tmpejj1q6i0/model/ with prefix 9a47ae9ab06f4948
[INFO 23-04-19 16:16:13.3044 EDT decision_forest.cc:660] Model loaded with 50 root(s), 2702 node(s), and 14 input feature(s).
[INFO 23-04-19 16:16:13.3046 EDT abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-04-19 16:16:13.3047 EDT kernel.cc:1074] Use fast generic engine
2023-04-19 16:16:13.325098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [699]
	 [[{{node Placeholder/_11}}]]


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [49]:
# Convert the testing data to a TensorFlow Dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data)

# Make predictions on the testing data
y_pred = model.predict(test_ds)

2023-04-19 16:16:14.569309: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype double and shape [175]
	 [[{{node Placeholder/_6}}]]


1/1 [==============================] - 0s 85ms/step


In [50]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=2)

In [51]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (14):
	Between
	CombWords
	Degree
	FamHist
	GCC
	GramComplex
	HC
	Level.ethnic
	Level.fed
	Level.gender
	Level.income
	Level.med
	PathLength
	Percentile

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.   "Percentile"  0.280091 ################
    2.    "CombWords"  0.236790 ##########
    3.  "GramComplex"  0.218871 ########
    4.          "GCC"  0.192062 #####
    5.           "HC"  0.189847 #####
    6.       "Degree"  0.188856 #####
    7.      "Between"  0.185558 ####
    8.   "PathLength"  0.183286 ####
    9.      "FamHist"  0.157640 #
   10.    "Level.fed"  0.155451 #
   11. "Level.income"  0.150453 
   12.  

In [52]:
model.compile(metrics=["accuracy"])
evaluation = model.evaluate(test_data, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

ValueError: in user code:

    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 518, in test_function  *
        return step_function_trained(self, iterator)
    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 497, in run_step  *
        outputs = model.test_step(data)
    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1788, in test_step  **
        y_pred = self(x, training=False)
    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filel1b_i87u.py", line 104, in tf__call
        ag__.if_stmt(ag__.ld(self)._semantics is None, if_body_2, else_body_2, get_state_3, set_state_3